# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [26]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [24]:
from pyspark.sql.functions import avg, min, max, count

In [27]:
filtered_df = df.filter((col("SALARY").isNotNull()) & (col("SALARY") != 0))

aggregated_df = filtered_df.groupBy("EMPLOYMENT_TYPE_NAME").agg(
    count("*").alias("job_count"),
    avg("SALARY").alias("avg_salary"),
    min("SALARY").alias("min_salary"),
    max("SALARY").alias("max_salary")
)

aggregated_df.show()



+--------------------+---------+------------------+----------+----------+
|EMPLOYMENT_TYPE_NAME|job_count|        avg_salary|min_salary|max_salary|
+--------------------+---------+------------------+----------+----------+
|Part-time / full-...|      619| 105621.2423263328|     20800|    455375|
|Part-time (â‰¤ 32...|     1038| 98802.50963391137|     15860|    310050|
|Full-time (> 32 h...|    29151|118897.55860862407|     20583|    500000|
+--------------------+---------+------------------+----------+----------+



In [4]:
filtered_df.select("EMPLOYMENT_TYPE_NAME", "SALARY") \
           .write.mode("overwrite") \
           .option("header", True) \
           .csv("filtered_salary_data")

import pandas as pd
import glob

files = glob.glob("filtered_salary_data/part-*.csv")
pdf = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)


In [33]:
pdf["EMPLOYMENT_TYPE_NAME"] = pdf["EMPLOYMENT_TYPE_NAME"].replace({
    "Part-time (â‰¤ 32 hours)": "Part-time (≤ 32 hours)",
    "Full-time (â‰¥ 32 hours)": "Full-time (≥ 32 hours)",
    "Part-time / full-time": "Part-time / full-time"
})


In [34]:
# Your Code for 1st question here
#pdf = df.select("EMPLOYMENT_TYPE_NAME", "SALARY").toPandas()
import plotly.io as pio
pio.renderers.default = "notebook_connected"

fig = px.box(pdf, x="EMPLOYMENT_TYPE_NAME", y="SALARY", 
             title="Salary Distribution by Employment Type", 
             color = "EMPLOYMENT_TYPE_NAME",
             color_discrete_sequence= ["yellow", "red", "coral"],
             labels={
        "EMPLOYMENT_TYPE_NAME": "Employment Type",
        "SALARY": "Salary"
    })
fig.update_layout(font_family="Aptos", title_font_size=17, 
                  xaxis=dict(title=dict(text='Employment Type'), zeroline=False),
                  yaxis=dict(title=dict(text='Salary'), zeroline=False))


fig.show()

First I cleaned up the employment type labels in the DataFrame for readability in the plot. From the box plot, we can observe that full-time employment has a significant number of outliers on the higher end of the salary distribution. In contrast, part-time (≤ 32 hours) shows a more tightly concentrated distribution with fewer extreme values. The interquartile range for part-time/full-time jobs appears narrower, indicating lower variability within that category. Also, the median salary for full-time roles is the highest among all groups, while part-time roles have the lowest median salaries.

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [6]:
# Your code for 2nd question here

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [7]:
# Your code for 3rd question here

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [8]:
# Your code for 4th question here

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [9]:
# Your code for 5th question here

# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [10]:
# Your code for 6th question here


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [11]:
# Your code for 7th question here

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [12]:
# Your code for 8th question here